# Non-negative Matrix Factorization using Alternating Least Squares
## Formulation
- Matrix Factorization (MF)
MF is a very commonly used technique in data compression, dimension reduction, clustering, and feature learning. Given a matrix $A \in \mathbb{R}^{m \times n}$, our task is to find a low rank approximation to $A$. The most popular formation is
$$
\min _{H \in \mathbb{R}^{m \times k}, W \in \mathbb{R}^{n \times k}} f(H, W):=\frac{1}{2}\left\|A-H W^{T}\right\|_{F}^{2} .
$$
If $A$ is the data matrix (each column is a data sample), then $H$ is usually explained as the pattern matrix and $W$ is the coefficient matrix. This problem in (1) is not convex but its global optimal solution can be obtained. 


- Alternating Least Squares 
1.1 Alternating Least Square method (ALS)
The ALS algorithm is a special case of the block coordinate descent algorithm. The key motivation behind of ALS is to iteratively optimize a single variable or a single group variable while fix the rest. To solve (1), we repeatedly solve $H$ fixing $W$ first and then solve $W$ fixing $H$. First we fix $W$ to solve $H$ by
$$
\min _{H \in \mathbb{R}^{m \times k}} \frac{1}{2}\left\|A-H W^{T}\right\|_{F}^{2} .
$$
This is a least squares problem. (We can solve it using the algorithms we learned in our last class.) Here, we give a closed form solution to it. To let the gradient be zero, we have
$$
\left(A-H W^{T}\right) W=0
$$


Similarly, we can solve $W$ by fixing $H$. We can have the closed form solution vt solving
$$
\left(A-H W^{T}\right)^{T} H=0 .
$$

The complexity of each iteration is $O(m n r+$ $\left.m r^{2}+n r^{2}\right)$.


----

## Extension
Further, we could regularize $H$ and $W$ using the L2 penalty. 
$$
\min _{H \in \mathbb{R}^{m \times r}, W \in \mathbb{R}^{n \times r}} f(H, W):=\frac{1}{2}\left\|A-H W^{T}\right\|_{F}^{2} + \lambda (\left\| H \right\|^2 + \left\| W \right\|^2) .
$$

Alternating Least Squares (ALS) 
1. Initialize the user vectors $H$ somehow (e.g. randomly).

2. If we use closed form solutions, repeate (a) and (b) until desired level of convergence is achieved. 

    (a) For each item i, let $A_{\cdot i}$ be the vector of ratings of that item (it will have m_users components; one for each user). Compute
    $$
    W_{i\cdot}=\left(H^{T} H+\lambda I\right)^{-1} H^{T} A_{\cdot i}
    $$
    for each item i. 

    (b) For each user u, let $A_{u \cdot}$ be the vector of ratings of that user (it will have n_items components; one for each item). Compute
    $$
    H_{u\cdot}=\left(W^{T} W+\lambda I\right)^{-1} W^{T} A_{u \cdot}
    $$
    for each user $\mathrm{u}$.

----
Alternatively, (a) and (b) could be substituted by the least-squares solution to a linear matrix equation. 



In [23]:
import numpy as np
from numpy.linalg import lstsq
import pandas as pd


def ALS_NMF(A, k, num_iter, init_H = None, init_W = None, print_enabled = True):
	'''
	Run the alternating least squares method to perform nonnegative matrix factorization on A.
	Return matrices W, H such that A = WH.
	
	Parameters:
		A: ndarray
			- m by n matrix to factorize
		k: int
			- integer specifying the column length of W / the row length of H
			- the resulting matrices W, H will have sizes of m by k and k by n, respectively
		num_iter: int
			- number of iterations for the multiplicative updates algorithm
		print_enabled: boolean
			- if ture, output print statements

	Returns:
		H: ndarray
			- m by k matrix where k = dim
		W: ndarray
			- n by k matrix where k = dim
		
	'''

	print('Applying the alternating least squares method on the input matrix...')

	if print_enabled:
		print('---------------------------------------------------------------------')
		print('Frobenius norm ||A - HW^{T}||_F')
		print('')

	# Initialize H and W

	if init_H is None:
		H = np.random.rand(np.size(A, 0), k )
	else:
		# you may check the shape of init_H before assignment 
		H = init_H

	if init_W is None:
		W = np.random.rand(np.size(A, 1), k)
	else:
		# you may check the shape of init_W before assignment 
		W = init_W


	# Decompose the input matrix
	for n in range(num_iter):
		# Update W
		# Solve the least squares problem: argmin_W^{T} ||HW^{T} - A||
		W = lstsq(H, A, rcond = -1)[0].T
		# Set negative elements of H to 0
		W[W < 0] = 0

	    # Update H
		# Solve the least squares problem: argmin_H.T ||WH^{T} - A.T||
		H = lstsq(W, A.T, rcond = -1)[0].T

		# Set negative elements of W to 0
		H[H < 0] = 0

		if print_enabled:
			frob_norm = np.linalg.norm(A - H @ W.T, 'fro')
			print("iteration " + str(n + 1) + ": " + str(frob_norm))

	return H, W

In [24]:
d = {'item0': [np.nan, 7, 6, 1, 1], 'item1': [3, 6, 7, 2, np.nan], 
      'item2': [3, 7, np.nan, 2, 1], 'item3': [1, 4, 4, 3, 2],
      'item4': [1, 5, 3, 3, 3], 'item5': [np.nan, 4, 4, 4, 3]}
df = pd.DataFrame(data=d).astype('float32')

X = np.array(df)
missing_idx = np.isnan(X)

X1 = X.copy()
X1[missing_idx] = 2

H, W = ALS_NMF(X1, k = 2, num_iter= 15)

Applying the alternating least squares method on the input matrix...
---------------------------------------------------------------------
Frobenius norm ||A - HW^{T}||_F

iteration 1: 5.202488752790232
iteration 2: 4.191993431343542
iteration 3: 4.113824059999216
iteration 4: 4.058007813906313
iteration 5: 4.013112225139666
iteration 6: 3.9802648988460585
iteration 7: 3.9579683344249985
iteration 8: 3.943595428406581
iteration 9: 3.9346366594549123
iteration 10: 3.9291691690889077
iteration 11: 3.9258752708494726
iteration 12: 3.9239063028768593
iteration 13: 3.9227348223534477
iteration 14: 3.9220397640407048
iteration 15: 3.921628056416324


# How to deal with missing data in $R$? 


In [25]:
import numpy as np
from numpy.linalg import lstsq
import pandas as pd
from numpy.linalg import lstsq


def ALS_NMF_missing(A, k, max_iter = 100, eps = 0.05, init_H = None, init_W = None, print_enabled = True):
	'''
	Run the alternating least squares method to perform nonnegative matrix factorization on A.
	Return matrices W, H such that A = WH.
	
	Parameters:
		A: ndarray
			- m by n matrix to factorize
		k: int
			- integer specifying the column length of W / the row length of H
			- the resulting matrices W, H will have sizes of m by k and k by n, respectively
		num_iter: int
			- number of iterations for the multiplicative updates algorithm
		print_enabled: boolean
			- if ture, output print statements

	Returns:
		H: ndarray
			- m by k matrix where k = dim
		W: ndarray
			- n by k matrix where k = dim
		
	'''

	print('Applying the alternating least squares method on the input matrix...')

	if print_enabled:
		print('---------------------------------------------------------------------')
		print('Frobenius norm ||A - HW^{T}||_F')
		print('')

	# Initialize H and W

	if init_H is None:
		H = np.random.rand(np.size(A, 0), k )
	else:
		# you may check the shape of init_H before assignment 
		H = init_H

	if init_W is None:
		W = np.random.rand(np.size(A, 1), k)
	else:
		# you may check the shape of init_W before assignment 
		W = init_W
	
	A_ori = A
	rowmean = np.nanmean(A, axis = 1)
	inds = np.where(np.isnan(A)) 
	#Place row means in the indices. Align the arrays using take
	A[inds] = np.take(rowmean, inds[0])
	# Decompose the input matrix
	A_est = np.matmul(H,W.T)
	sq_error = np.square(np.subtract(A, A_est)) 
	sq_error[inds] = 0
	loss_pre = np.sqrt(sq_error.sum())

	# Decompose the input matrix
	diff = 10000
	count = 0
	while diff > eps: 
		if count > max_iter: 
			print('Maximum Iteration Reached!! ')
			break 
			# Update W
			# Solve the least squares problem: argmin_W^{T} ||HW^{T} - A||
		W = lstsq(H, A, rcond = -1)[0].T
		# Set negative elements of H to 0
		W[W < 0] = 0

		# Update H
		# Solve the least squares problem: argmin_H.T ||WH^{T} - A.T||
		H = lstsq(W, A.T, rcond = -1)[0].T

		# Set negative elements of W to 0
		H[H < 0] = 0

		### Calculate the squared error for only for the existing entries. 
		A_est = np.matmul(H,W.T)
		sq_error = np.square(np.subtract(A, A_est)) 
		### ignore missing data
		sq_error[inds] = 0
		sq_error_sum = np.sqrt(sq_error.sum()) 
		if print_enabled:
			print("iteration " + str(count + 1) + ": " + str(sq_error_sum))
		diff = np.absolute(sq_error_sum - loss_pre)
		loss_pre = sq_error_sum
		A[inds] = A_est[inds]
		count += 1
	return H, W


In [27]:

d = {'item0': [np.nan, 7, 6, 1, 1], 'item1': [3, 6, 7, 2, np.nan], 
      'item2': [3, 7, np.nan, 2, 1], 'item3': [1, 4, 4, 3, 2],
      'item4': [1, 5, 3, 3, 3], 'item5': [np.nan, 4, 4, 4, 3]}
df = pd.DataFrame(data=d).astype('float32')

X = np.array(df)
X
W, H  = ALS_NMF_missing(X, k=2, max_iter = 50, eps = 0.00005)


Applying the alternating least squares method on the input matrix...
---------------------------------------------------------------------
Frobenius norm ||A - HW^{T}||_F

iteration 1: 3.9691448846081516
iteration 2: 2.6617188771030116
iteration 3: 2.1282095489096986
iteration 4: 1.9047445523034296
iteration 5: 1.81876365332908
iteration 6: 1.7862897053781979
iteration 7: 1.773050979197988
iteration 8: 1.7672225102122554
iteration 9: 1.7645472884311624
iteration 10: 1.7632993243906623
iteration 11: 1.762714503612727
iteration 12: 1.7624403098407593
iteration 13: 1.7623118072752912
iteration 14: 1.7622515967805634
iteration 15: 1.7622233746822888
